# Micrograph Clustering With Karas


## Abstract
   Machine learning is the scientific study of algorithms and statistical models that computer systems use to perform tasks.  Machine learning can be used to analyze many different types of data but the use of machine learning for electron microscopy images looked at for this paper. There are two main methods for image analysis being classification and clustering. The classification of images uses pre-labeled datasets to train a model so that unlabeled datasets can be sorted into their respective categories. This is what can be considered supervised learning because the user must train the model prior to classification. The  other method of image analysis is clustering.  This is a method of unsupervised learning in which images are sorted based off of similarity.  For the purpose of our project we chose to use image clustering with VGG16 and VGG19. Due to our large set of unknown microscopy images it made more sense to have the computer sort images rather than personally going through and labeling images individually so that a network could be trained. We decided to look at the differences in how VGG16 and VGG19 would classify our images. We also looked at how population size and number of clusters affect accuracy using VGG19. Looking at our data we were able to see a clear improvement from VGG16 to VGG19. We also saw better accuracy when using more clusters on our full dataset. We were also able to conclude that by refining...

## Introduciton
Image Classification is a type of machine learning and can be tested with different types of models. Although there are different types of models, all can be used to compare properties of an image against other images. These models can compare information from the size of the image to the color of each pixel. With this in mind, we compiled our images into one location and began researching the perfect model for our problem. Our goal was to create a code that would sort and classify the images together based on their similarities. We first tested our images using Principal Component Analysis (PCA). PCA works by pulling out features from the data without losing information and compares them against each other. We saw however, that the images had a high dimensionality and could not be sorted by PCA. Our second test was to use Fast.Ai. Fast.Ai works similarly to PCA by pulling out the features, but compares and sorts them based on given parameters. When testing with Fast.Ai we ran into issues because we are comparing the images against themselves instead of giving it parameters to compare them by. Lastly we came across another machine learning program called Keras. This program contains two famous models known as VGG16 and VGG19. These models work by pulling out the features from each of the images and then compares each of the features to all the images. The images with similar features get grouped together into what is known as a cluster. We lastly noticed that VGG19 achieved better results and clustered the images more efficiently than VGG16.

## Results and Discussion

### Our Code
Prior to starting on the code the propper Nvidea GPU drivers and cuDNN were installed on the computers. Inorder for Keras to run efficiently on a gpu it requires a backend classifier known as TensorFlow-Gpu. Once Keras and TensorFlow-Gpu were installed onto anaconda, we began our testing by importing the libraries:

In [ ]:
from keras.preprocessing import image
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
import numpy as np
from sklearn.cluster import KMeans
import os, shutil, glob, os.path
from PIL import Image as pil_image
import matplotlib.pyplot as plt

We then imported and loaded our VGG19 Model. This was followed by us displaying the models layers to ensure that VGG19 had loaded successfully. It is important to note that by setting load truncated images to false we did not allow for images to be compressed by an uneven ratio in hopes for higher accuracy.

In [ ]:
image.LOAD_TRUNCATED_IMAGES = False 
model = VGG19(weights='imagenet', include_top=False)
model.summary()

The output shows the summary of the model, which displays the amount of layers and how many parameters each layer has. Next we locate the directory for our images, as well as define a location to save the classified images. We also define the amount of clusters the images will be placed into:

In [ ]:
imdir = 'A:\School\Mat 397\Smeltzer STEM\TEM'
targetdir = 'A:\School\Mat 397\Smeltzer STEM\TEM\Tem Sorted\Tem Sorted'
number_clusters = 5

The images were converted from DM3 files from different TEM microscopes taken by various people and thus are varied in size. Our next line of code loads in the images into a data bunch in order for VGG to be run. While loading in the data the images are reduced to a size of 224x224 which is required for VGG to run. The images are then loaded into our model at which point VGG essentially pulls out and defines features from each image:

In [ ]:
filelist = glob.glob(os.path.join(imdir, '*.png'))
filelist.sort()
featurelist = []
for i, imagepath in enumerate(filelist):
    print("    Status: %s / %s" %(i, len(filelist)), end="\r")
    img = image.load_img(imagepath, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    features = np.array(model.predict(img_data))
    featurelist.append(features.flatten())

Now in order to analyze and visualize this data we used a model from SKlearn known as KMeans. KMeans takes the features from the VGG19 test and compares and clusters them together in the number of clusters previously defined:

In [ ]:
kmeans = KMeans(n_clusters=number_clusters, random_state=0).fit(np.array(featurelist))

The images are now in five clusters and must be saved back to a directory so that they may be viewed. This code saves the images into the desired directory and gives them a name based on their cluster:

In [ ]:
print("\n")
for i, m in enumerate(kmeans.labels_):
    print("    Copy: %s / %s" %(i, len(kmeans.labels_)), end="\r")
    shutil.copy(filelist[i], 'A:\School\Mat 397\Smeltzer STEM\TEM\Tem Sorted\Tem Sorted' + str(m) + "_" + str(i) + ".png")

Lastly we graph the KMeans clusters and cluster centers and the to help visualize the variance in each of the clusters:

In [ ]:
plt.scatter(
    kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,4],
    s=250, marker='.',
    c='red', edgecolor='black',
    label='centroids'
)

### Analysis
To begin with we started with VGG16 to establish a baseline of how accurate the image clustering would be. We choose VGG16 as a baseline due to its speed and fewer layers compared to the other karas models. We ran the model several times using three four and five clusters and concluded using five clusters yielded the most accurate image clustering. As seen in the following images cluster one and cluster two seem to be grouped on average contrast rather than features. These clusters housed a majority of the images from our dataset and also showed the biggest spread of image similarity. 

![Figure 1](https://raw.githubusercontent.com/myi220/Mat-397-project/master/pictures%20(1).jpg)
**<center>Figure 1 | Image Clusters Using VGG16 and KMeans with Five Clusters** <center> **a,** First Cluster **b,** Second Cluster **c,** Third Cluster **c,** Fourth Cluster **d,** Fifth Cluster

Now looking at clusters three and four both display boundaries with a darker region on one side and a lighter region on the other. While these images are images are classified very well, with no outliers in either cluster, they should have been clustered together. The only distinction between these two clusters is the contrast, cluster three having a higher contrast than cluster four. Cluster five was overall the most successful cluster. This section solely included images of crystal lattices with no outliers. While images that should have belonged in this group were spread out among all the other clusters this was predominantly in the case of more complex images.

The KMeans average and spread can be seen in the following diagrams. This shows how the computer differentiated the images compared to each other. The closer the points the fewer differences the computer saw between the images. The right two most points seen on the KMeans cluster center diagrams include a large range of images and represent clusters one and two. The points that are overlapping from this diagram are clusters three and four. The small difference between these points validates what we saw by eye that these clusters are incredibly similar. Finally the bottom left most point shows cluster five. Due to the large spread of images four of the KMeans cluster centers appear close to one another. This is due to the lone KMeans cluster center which is an average of all the miscellaneous images.

![Figure 2](https://raw.githubusercontent.com/myi220/Mat-397-project/master/pictures%20(3).jpg)
**<center>Figure 2 | KMeans Cluster Centers Graphed VGG16 (left) and Simularity of Images Graphed VGG16 (right)<center>** 

We were overall not incredibly thrilled with the results from the VGG16 test so with a few changes we ran VGG19 over the same images to see the differences between the two models. To keep things consistent, for comparing the two models, we used five clusters. Right off the bat it was apparent that clustering using VGG19 had yielded more accurate results. These images seen in clusters one through four all seem to be similar. It seems as if VGG19 clustered the images more based off of features rather than contrast as in VGG16. While cluster four had many similar images it is the first cluster that outlier images start being seen in.

![Figure 3](https://raw.githubusercontent.com/myi220/Mat-397-project/master/VGG19%20Full%20Photos.png)
**<center>Figure 3 | Image Clusters Using VGG19 and KMeans with Five Clusters** <center> **a,** First Cluster **b,** Second Cluster **c,** Third Cluster **c,** Fourth Cluster **d,** Fifth Cluster

The final cluster, cluster five, contains images with the same relative grey scale. While the images aren't necessarily similar in features they do for the most part lack the crispness seen in other images. It seems as if for this cluster the feature the model detected in the images was areas of low focus.

Below in Figure 4 the image similarity spread and KMeans cluster centers can be seen. The cluster centers are significantly more spread out than those from VGG16. This is apparent when looking through the image clusters. In general the images share more similarity within clusters than seen previously. The image similarity spread has also changed due to the additional layers that VGG19 employs. While there are outlying images within each cluster or images that belong in other clusters the overall accuracy is significantly better than what was seen using VGG16.

![Figure 4](https://raw.githubusercontent.com/myi220/Mat-397-project/master/pictures%20(5).jpg)
**<center>Figure 4 | KMeans Cluster Centers Graphed VGG19 (left) and Simularity of Images Graphed VGG19 (right)<center>** 

The last thing we decided to test was how the model would be affected by population size. This was done by taking 100 images from three of the more accurate clusters returned in VGG19. For this population size the model was run with three clusters rather than five. The images, seen below in Figure 5, from each of the clusters show higher accuracy compared to what was seen in the previous two tests. This however was in part due to the images already having three groups rather than being completely random. This did show us that VGG19 could still be accurately used on smaller image databases.

![Figure 5](https://raw.githubusercontent.com/myi220/Mat-397-project/master/pictures%20(4).jpg)
**<center>Figure 5 | Image Clusters Using VGG19 and KMeans with Three Clusters** <center> **a,** First Cluster **b,** Second Cluster **c,** Third Cluster

One interesting thing we found was that when running the model over again on these 300 images some images changed clusters. Images that we recognized prior that should have belonged in different clusters were changed to fit in the similar cluster.  The big reason for this is due to the images having more distinct categories going into clustering. This makes sense because of our large number of random TEM images many were distinct (and shouldnt belong in a cluster) which would vary how the KMeans averages and clusters images together. While our project yielded impressive results it would be recommended that if attempted again distinct images be removed to improve the accuracy of the clusters.

## Conclusion
Overall VGG19 had the best results in resizing and defining the features in each image. VGG16 achieved mediocre results and tended to cluster images without similar features together. This is most likely due to VGG19 having more layers and parameters than VGG16. These extra layers allow the model to analyze more features in the images. Inorder to cluster the images we used the KMeans model. We noticed that doing KMeans with more than 3 clusters resulted in better grouping however we also saw that doing KMeans with too many clusters leads to grouping of non similar images. When testing KMeans with different sizes of data we concluded that as the amount of images increased, the efficiency of the KMeans clustering increased as well.

## References
1.  Elsinghorst, Shirin. “Image Clustering with Keras and k-Means.” Shirin's PlaygRound, 6 Oct. 2018, shirinsplayground.netlify.com/2018/10/keras_fruits_cluster/.
2.  “Image Clustering by Its Similarity in Python.” Image Clustering by Its Similarity in Python, 2016, doi:10.1007/978-1-4842-2388-8_6.
3.  “Python Keras.applications.vgg16.VGG16() Examples.” Program Creek, programcreek.com/python/example/92213/keras.applications.vgg16.VGG16.
4.  Chm. “Visualize Image Clustering.” Mc.ai, 30 Mar. 2018, mc.ai/visualize-image-clustering/.
5.  Franky. “Using Keras' Pre-Trained Models for Feature Extraction in Image Clustering.” Medium, Medium, 5 Apr. 2018, medium.com/@franky07724_57962/using-keras-pre-trained-models-for-feature-extraction-in-image-clustering-a142c6cdf5b1.